# Importing all the necessairy packages and initialising our identity

The SEC wants us to tell them who we are in order to process our request.

In [327]:
import pandas as pd
import yfinance as yf
import numpy as np
from bs4 import BeautifulSoup
import requests
import json
import re
import time
import difflib

# This part I want to figure out if I can get the data from the json files of SEC API.

https://www.sec.gov/edgar/sec-api-documentation

This API is created by the SEC to make finding the data we want much easier. It uses **XBRL** data APIs, starting from 2009 which will give the data some structure. 

The thing about this API is, that the data is structure in a json file. For Python this means there are a lot of nested dictionaris. So we will need to find the right data, inside the nested environment



In [328]:
#set my identity for the SEC (First name last name and email)
name = "Maseeh"
surname = "Faizan"
email = "maseehfaizan@gmail.com"
headers = {'User-Agent':f'{name} {surname} {email}'}

#### First how ever we need to scrape all of the company information like:
- Name
- Ticker
- cik (unique identifier code). cik code needs to be **10 digits with leading zeros**

In what follows I will define a pandas dataframe *ticker_df* to have all of the information on the identifier in the same place

In [329]:
ticker = requests.get('https://www.sec.gov/files/company_tickers.json',headers=headers).json()
ticker_df = pd.DataFrame.from_dict(ticker,orient='index')
ticker_df.rename(columns={'cik_str':'cik','title':'name'},inplace=True)
#Filing in the cik code and adding the leading zeros
ticker_df['cik'] = ticker_df['cik'].astype(str).str.zfill(10)
ticker_df.head()

,cik,ticker,name
0,0000789019,MSFT,MICROSOFT CORP
1,0000320193,AAPL,Apple Inc.
2,0001045810,NVDA,NVIDIA CORP
3,0001652044,GOOGL,Alphabet Inc.
4,0001018724,AMZN,AMAZON COM INC


### Now I need to ask for the ticker name and with that will find out the cik name and everything else
##### Moreover, I need to add leading zeros making the cik number a total of 10, as suggested in the documentation

In [403]:
# I need to find the cik code given the company ticker lets take AAPL as an example 
tic = 'MSFT'
cik = ticker_df.loc[ticker_df['ticker'] == tic, 'cik'].iloc[0]
print(cik)

0000789019


### First let's get our data given the cik value of the company. 

- **submission file**, contains all of the meta data 
Let's see what is in the submission files

It apears to be an embeded dictionary with different infrormation.
After some random information, like website and category we have filings, and in the key of filings there is an embeded dictionary, in which we will find further information about the accounts we are looking for

Let's see what is inside it:


In [404]:
files = requests.get(f'https://data.sec.gov/submissions/CIK{cik}.json',headers=headers).json()
files.keys()

dict_keys(['cik', 'entityType', 'sic', 'sicDescription', 'insiderTransactionForOwnerExists', 'insiderTransactionForIssuerExists', 'name', 'tickers', 'exchanges', 'ein', 'description', 'website', 'investorWebsite', 'category', 'fiscalYearEnd', 'stateOfIncorporation', 'stateOfIncorporationDescription', 'addresses', 'phone', 'flags', 'formerNames', 'filings'])

We fine a column of recent and a column of files and recent. 
- Files is a dead end. 
For now I want to look into the recent submissions and try to turn the dictionary into Pandas DataFrame. This contain the file, number the kind of form it is, accessionNumber code. Handy information to move forward with

In [405]:
files['filings'].keys()

dict_keys(['recent', 'files'])

In [406]:
files['filings']['recent'].keys()

dict_keys(['accessionNumber', 'filingDate', 'reportDate', 'acceptanceDateTime', 'act', 'form', 'fileNumber', 'filmNumber', 'items', 'size', 'isXBRL', 'isInlineXBRL', 'primaryDocument', 'primaryDocDescription'])

### Here I have all of the forms and I only need the 10Ks for now to find the Balance Sheet and other Financial statemnts. 

I am taking all the recent submissions and turning it into a **Pandas Dataframe** then I specifically want the **10K** to be used as anual information. 

I now have information like
- 10-K form
- filing date
- it's primary Document
- XBRL information


In [432]:
recent_files = pd.DataFrame(files['filings']['recent'])
tenk =recent_files[recent_files['form'] == '10-K']
tenk.head(3)

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
125,0000950170-23-035122,2023-07-27,2023-06-30,2023-07-27T16:01:56.000Z,34,10-K,001-37845,231118330,,40966463,1,1,msft-20230630.htm,10-K
267,0001564590-22-026876,2022-07-28,2022-06-30,2022-07-28T16:06:19.000Z,34,10-K,001-37845,221115247,,26423137,1,1,msft-10k_20220630.htm,10-K
408,0001564590-21-039151,2021-07-29,2021-06-30,2021-07-29T16:21:55.000Z,34,10-K,001-37845,211127769,,26907098,1,1,msft-10k_20210630.htm,10-K


### Now that I have all this information, I can simply use them to find the kind of data I am actually looking for (Financial information)

- I will use the data in the dataframe above and find the company facts that is organised using the **SEC XBRL *API*** 

Here we have three keys, 
- cik 
- company name 
- facts. 

Let's look into facts

In [425]:
facts = requests.get(f'https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json',headers = headers).json()
facts.keys()

dict_keys(['cik', 'entityName', 'facts'])

#### In company's facts we have dei code and US-GAAP. In dei we have public shares outstanding data. What we are interested in is us-gaap which has list of all the balance sheed and income statement information

In [433]:
facts['facts'].keys()

dict_keys(['dei', 'us-gaap'])

In [446]:
facts['facts']['us-gaap'].keys()

dict_keys(['AccountsPayable', 'AccountsPayableCurrent', 'AccountsReceivableNetCurrent', 'AccruedIncomeTaxesCurrent', 'AccruedIncomeTaxesNoncurrent', 'AccruedLiabilities', 'AccruedLiabilitiesCurrent', 'AccruedMarketingCostsCurrent', 'AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment', 'AccumulatedOtherComprehensiveIncomeLossAvailableForSaleSecuritiesAdjustmentNetOfTax', 'AccumulatedOtherComprehensiveIncomeLossCumulativeChangesInNetGainLossFromCashFlowHedgesEffectNetOfTax', 'AccumulatedOtherComprehensiveIncomeLossForeignCurrencyTranslationAdjustmentNetOfTax', 'AccumulatedOtherComprehensiveIncomeLossNetOfTax', 'AdjustmentsToAdditionalPaidInCapitalSharebasedCompensationRequisiteServicePeriodRecognitionValue', 'AdjustmentsToAdditionalPaidInCapitalTaxEffectFromShareBasedCompensation', 'AdvertisingExpense', 'AllocatedShareBasedCompensationExpense', 'AllowanceForDoubtfulAccountsReceivableCurrent', 'AmortizationOfIntangibleAssets', 'AntidilutiveSecuritiesExcludedFromCompu

Here I am calling the variable **accounts** the name of the accounts that are at my disposal in the SEC.

- I am going to normalize the naming process. Remove any capital letter, dots or spaces there might be. 

- I am then going to create a dictionary. The **keys** are the name of the accounts and the **value** will be the name that is present are present in most of the **10K**. There are different accounts and different naming schemes for the same accounts given different countries. My code needs to work for most of the companies. I have gone through dosens of them and have come up with the most common naming schemes.


Furthermore, I am also taking the most common accounts, like Assets, Liability, Revenues COGS. These are the accounts we care about the most and these are also accounts that are commun accross different companies.


I have shyed away from specific accounts, like Amazon has Fulfillment account, Apple has PPE, but then Pfizer has very different accounts. I can't possibliy account for all those different variation **FOR NOW** 

In [ ]:

accounts = list(facts['facts']['us-gaap'].keys())

# Normalize everything down
normalized_account = [re.sub(r'\W+', '',s).lower() for s in accounts]

names = {'Assets':['Assets'],
        'Current Assets':['assets current'],
        'Cash':['Cash and cash equivalents'],
        'Current Liability':['Liability current'],
        'Total Stockholders equity':['Stockholders Equity'],
        'Liability and Stockholder equity':['Liabilities and Stockholders equity'],
        'Revenues':['Sales'],
        'COGS':['Cost of Goods Sold'],
        'Operating Expense':['Total Operating Expense'],
        'Net Income':['Net Income'],
        'EPS':['Earnings Per Share basic']
        }

### Before matching both lists []

#### Finding the best match and putting it into a new finalized dictionary

### Different naming schemes for different accounts

### Now that I have all that information I will start off by finding the necessary information for the most recent accounts.

Here I have the information for all of the companie's accounts. 

Let's create a new DataFrame off of that

# Let's get the balance sheet done

In [424]:



name = "Maseeh"
surname = "Faizan"
email = "maseehfaizan@gmail.com"
headers = {'User-Agent':f'{name} {surname} {email}'}


ticker = requests.get('https://www.sec.gov/files/company_tickers.json',headers=headers).json()
ticker_df = pd.DataFrame.from_dict(ticker,orient='index')
ticker_df.rename(columns={'cik_str':'cik','title':'name'},inplace=True)
#Filing in the cik code and adding the leading zeros
ticker_df['cik'] = ticker_df['cik'].astype(str).str.zfill(10)

tic = 'AAPL'
cik = ticker_df.loc[ticker_df['ticker'] == tic, 'cik'].iloc[0]


facts = requests.get(f'https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json',headers = headers).json()


accounts = list(facts['facts']['us-gaap'].keys())

# Normalize everything down

normalized_account = [re.sub(r'\W+', '',s).lower() for s in accounts]

names = {'Assets':['Assets'],
        'Current Assets':['assets current'],
        'Cash':['Cash and cash equivalents'],
        'Current Liability':['Liability current'],
        'Total Stockholders equity':['Stockholders Equity'],
        'Liability and Stockholder equity':['Liabilities and Stockholders equity'],
        'Revenues':['Sales'],
        'COGS':['Cost of Goods Sold'],
        'Operating Expense':['Total Operating Expense'],
        'Net Income':['Net Income'],
        'EPS':['Earnings Per Share basic'],

        }


# find the best match between the names dictionary and the list I have in the accounts
def find_best_match(items, search):
    matches = difflib.get_close_matches(search, items, n=1, cutoff=0.0)
    return matches[0] if matches else None

for key, value in names.items():
    value = [re.sub(r'\W+', '',s).lower() for s in value]
    best_match = [find_best_match(normalized_account, j) for j in value]
    
    accounts_match = [find_best_match(accounts, best_match[0])]
    names[key] = accounts_match[0]



balance = pd.DataFrame(columns=['Year','val'])
ix = 0
for key,value in names.items():
    # To comply with SEC we need to set a timer so the program doesn't request too many items at once
    time.sleep(0.1)
    name = key
    company_concept = requests.get(f'https://data.sec.gov/api/xbrl/companyconcept/CIK{cik}/us-gaap/{value}.json',headers=headers).json()
    # the unit for Earning per share is USD/Share and not USD which means I need to adapt the code for that
    if key == 'EPS':
        name = pd.DataFrame(company_concept['units']['USD/shares'])
    else:
        name = pd.DataFrame(company_concept['units']['USD'])
    name = name[name['form'] == '10-K']
    name = name.drop_duplicates(subset='fy',keep='last')
    name = name.tail(60)
    name = name[['fy','val']]
    name = name.rename(columns={'fy':'Year','val':key})
    # I want to initialize the balance sheet that is why I need to merge on name, then after I will merge on Balance Sheet dataframe
    if ix == 0:
        balance = balance.merge(name,left_on='Year',right_on='Year',how='right')
        ix +=1
    else:
        balance = balance.merge(name,left_on='Year',right_on='Year',how='left')

balance = balance.drop('val',axis = 1)
balance = balance.sort_values(by='Year',ascending=False)
balance = balance.set_index('Year')
balance = balance.T

balance

Year,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009
Assets,3.525830e+11,3.527550e+11,3.510020e+11,3.238880e+11,3.385160e+11,3.657250e+11,3.753190e+11,3.216860e+11,2.904790e+11,2.318390e+11,2.070000e+11,1.760640e+11,1.163710e+11,7.518300e+10,5.385100e+10
Current Assets,1.435660e+11,1.354050e+11,1.348360e+11,1.437130e+11,1.628190e+11,1.313390e+11,1.286450e+11,1.068690e+11,8.937800e+10,6.853100e+10,7.328600e+10,5.765300e+10,4.498800e+10,4.167800e+10,3.626500e+10
Cash,2.996500e+10,2.364600e+10,3.494000e+10,3.801600e+10,4.884400e+10,2.591300e+10,2.028900e+10,2.048400e+10,2.112000e+10,1.384400e+10,1.425900e+10,1.074600e+10,9.815000e+09,1.126100e+10,5.263000e+09
Current Liability,1.453080e+11,1.539820e+11,1.254810e+11,1.053920e+11,1.057180e+11,1.168660e+11,1.008140e+11,7.900600e+10,8.061000e+10,6.344800e+10,4.365800e+10,3.854200e+10,2.797000e+10,2.072200e+10,1.928200e+10
Total Stockholders equity,6.214600e+10,5.067200e+10,6.309000e+10,6.533900e+10,9.048800e+10,1.071470e+11,1.340470e+11,1.282490e+11,1.193550e+11,1.115470e+11,1.235490e+11,1.182100e+11,7.661500e+10,4.779100e+10,2.783200e+10
Liability and Stockholder equity,3.525830e+11,3.527550e+11,3.510020e+11,3.238880e+11,3.385160e+11,3.657250e+11,3.753190e+11,3.216860e+11,2.904790e+11,2.318390e+11,2.070000e+11,1.760640e+11,1.163710e+11,7.518300e+10,5.385100e+10
Revenues,NaN,NaN,NaN,NaN,NaN,NaN,5.257900e+10,4.685200e+10,5.150100e+10,4.212300e+10,3.747200e+10,3.596600e+10,2.827000e+10,2.034300e+10,3.653700e+10
COGS,2.141370e+11,2.235460e+11,2.129810e+11,1.695590e+11,1.617820e+11,1.637560e+11,1.410480e+11,1.313760e+11,1.400890e+11,1.122580e+11,1.066060e+11,8.784600e+10,6.443100e+10,3.954100e+10,2.339700e+10
Operating Expense,5.484700e+10,5.134500e+10,4.388700e+10,3.866800e+10,3.446200e+10,3.094100e+10,2.684200e+10,2.423900e+10,2.239600e+10,1.803400e+10,1.530500e+10,1.342100e+10,1.002800e+10,7.299000e+09,5.482000e+09
Net Income,9.699500e+10,9.980300e+10,9.468000e+10,1.267300e+10,1.368600e+10,1.412500e+10,1.071400e+10,9.014000e+09,1.112400e+10,8.467000e+09,7.512000e+09,8.223000e+09,6.623000e+09,4.308000e+09,5.704000e+09
